El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [34]:
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor
import lightgbm as lgb

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder


from xgboost import XGBRegressor



In [3]:
# Cargar el archivo CSV proporcionado
file_path = '/datasets/car_data.csv'
car_data = pd.read_csv(file_path)

In [11]:
# Visualizar las primeras filas del dataset y la información general
# Visualizar las primeras filas del dataset y la información general
car_data_info = car_data.info()
car_data_head = car_data.head()

car_data_info, car_data_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

(None,
         DateCrawled  Price VehicleType  RegistrationYear Gearbox  Power  \
 0  24/03/2016 11:52    480         NaN              1993  manual      0   
 1  24/03/2016 10:58  18300       coupe              2011  manual    190   
 2  14/03/2016 12:52   9800         suv              2004    auto    163   
 3  17/03/2016 16:54   1500       small              2001  manual     75   
 4  31/03/2016 17:25   3600       small              2008  manual     69   
 
    Model  Mileage  RegistrationMonth  FuelType       Brand NotRepaired  \
 0   golf   150000                  0    petrol  volkswagen         NaN   
 1    NaN   125000                  5  gasoline        audi         yes   
 2  grand   125000                  8  gasoline        jeep         NaN   
 3   golf   150000                  6    petrol  volkswagen          no   
 4  fabia    90000                  7  gasoline       skoda          no   
 
         DateCreated  NumberOfPictures  PostalCode          LastSeen  
 0  24/03/20

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Excelente trabajo en la preparación de los datos. Has identificado y eliminado columnas irrelevantes y manejado los valores faltantes de manera adecuada. Esto es crucial para garantizar la calidad del modelo.
</div>


In [12]:
# 1. Eliminar columnas irrelevantes para la predicción, como las fechas y el código postal
car_data_cleaned = car_data.drop(columns=['DateCrawled', 'DateCreated', 'LastSeen', 'PostalCode', 'NumberOfPictures'])

In [13]:
# 2. Remover filas donde el precio sea cero o extremadamente bajo (por ejemplo, menos de 100), ya que estos son probablemente errores
car_data_cleaned = car_data_cleaned[car_data_cleaned['Price'] >= 100]


In [14]:
# 3. Eliminar registros donde la potencia del coche sea cero, lo cual es irreal para la predicción de precio
car_data_cleaned = car_data_cleaned[car_data_cleaned['Power'] > 0]

In [15]:
# 4. Manejar valores faltantes
# Rellenar valores faltantes de las columnas categóricas con 'unknown'
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']
car_data_cleaned[categorical_columns] = car_data_cleaned[categorical_columns].fillna('unknown')


<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Aunque eliminaste las columnas 'DateCrawled', 'DateCreated' y 'LastSeen', considera que podrían contener información útil, como el tiempo que el anuncio estuvo activo, lo cual podría influir en el precio. Podrías explorar si esta información mejora el modelo.
</div>


<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Al eliminar registros con 'Price' menor a 100 y 'Power' igual a cero, estás manejando algunos valores atípicos. Sin embargo, podría ser útil realizar un análisis más profundo de los outliers en variables como 'RegistrationYear' y 'Power' para identificar y manejar otros valores atípicos que puedan afectar el modelo.
</div>


In [17]:
# Revisar nuevamente las primeras filas y la descripción general del dataset limpio
car_data_cleaned_info = car_data_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 305993 entries, 1 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              305993 non-null  int64 
 1   VehicleType        305993 non-null  object
 2   RegistrationYear   305993 non-null  int64 
 3   Gearbox            305993 non-null  object
 4   Power              305993 non-null  int64 
 5   Model              305993 non-null  object
 6   Mileage            305993 non-null  int64 
 7   RegistrationMonth  305993 non-null  int64 
 8   FuelType           305993 non-null  object
 9   Brand              305993 non-null  object
 10  NotRepaired        305993 non-null  object
dtypes: int64(5), object(6)
memory usage: 28.0+ MB


In [18]:
car_data_cleaned.head(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes
6,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no
8,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,unknown
9,999,small,1998,manual,101,golf,150000,0,unknown,volkswagen,unknown
10,2000,sedan,2004,manual,105,3_reihe,150000,12,petrol,mazda,no
11,2799,wagon,2005,manual,140,passat,150000,12,gasoline,volkswagen,yes


## Entrenamiento del modelo 

In [20]:
# Separar características (X) y objetivo (y)
X = car_data_cleaned.drop(columns=['Price'])
y = car_data_cleaned['Price']



In [21]:
# Codificar las características categóricas
categorical_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
encoder = OrdinalEncoder()
X[categorical_features] = encoder.fit_transform(X[categorical_features])



In [22]:
# Dividir el conjunto de datos en entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [23]:
# Mostrar la forma de los conjuntos de entrenamiento y prueba
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((214195, 10), (91798, 10), (214195,), (91798,))

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Dividiste el conjunto de datos en entrenamiento y prueba de manera adecuada, lo cual es fundamental para evaluar el rendimiento real del modelo en datos no vistos.
</div>


<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Has utilizado `OrdinalEncoder` para codificar las variables categóricas. Sin embargo, este método asigna un orden arbitrario a las categorías, lo cual puede no ser apropiado si no existe una relación ordinal entre ellas. Considera usar `OneHotEncoder` para variables categóricas nominales o aprovechar las capacidades de CatBoost y LightGBM para manejar variables categóricas directamente.
</div>


## Análisis del modelo

In [25]:
# Entrenar el modelo de regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_linear = linear_model.predict(X_test)

# Calcular el RMSE para la regresión lineal
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_linear))
rmse_linear

3614.146358501632

El modelo de regresión lineal sirvió como una prueba de cordura inicial y nos proporcionó un RMSE de 3614.15, lo que es razonable para un modelo lineal en este tipo de problemas.
Al probar otros modelos más avanzados como LightGBM, CatBoost, y XGBoost, es probable que el RMSE disminuya, lo que indica una mejora en la calidad de las predicciones. Si alguno de estos modelos tiene un RMSE significativamente peor que la regresión lineal, sería una señal de que algo no está funcionando correctamente.


In [31]:

# Crear el dataset de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Definir los parámetros del modelo
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.8,
    'random_state': 42
}

# Entrenar el modelo LightGBM
lightgbm_model = lgb.train(params, train_data, valid_sets=[train_data, test_data], verbose_eval=50)

# Predecir en el conjunto de prueba
y_pred_lgbm = lightgbm_model.predict(X_test)

# Calcular el RMSE para el modelo LightGBM
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))
rmse_lgbm

/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004932 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 678
[LightGBM] [Info] Number of data points in the train set: 214195, number of used features: 10
[LightGBM] [Info] Start training from score 4833.365107
[50]	training's rmse: 1771.43	valid_1's rmse: 1801.92
[100]	training's rmse: 1677.57	valid_1's rmse: 1715.88


1715.8793419801023

In [30]:

# Definir el modelo de CatBoost
catboost_model = CatBoostRegressor(loss_function='RMSE', random_state=42, verbose=100)

# Entrenar el modelo de CatBoost
catboost_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_catboost = catboost_model.predict(X_test)

# Calcular el RMSE para el modelo CatBoost
rmse_catboost = np.sqrt(mean_squared_error(y_test, y_pred_catboost))
rmse_catboost

Learning rate set to 0.095598
0:	learn: 4311.2460167	total: 24.5ms	remaining: 24.4s
100:	learn: 1821.7389156	total: 2.31s	remaining: 20.5s
200:	learn: 1728.7632829	total: 4.56s	remaining: 18.1s
300:	learn: 1681.9266387	total: 6.79s	remaining: 15.8s
400:	learn: 1647.7603980	total: 9.04s	remaining: 13.5s
500:	learn: 1622.7143848	total: 11.3s	remaining: 11.2s
600:	learn: 1601.7774865	total: 13.7s	remaining: 9.07s
700:	learn: 1583.2673191	total: 16s	remaining: 6.84s
800:	learn: 1567.6953532	total: 18.3s	remaining: 4.54s
900:	learn: 1555.2717206	total: 20.5s	remaining: 2.25s
999:	learn: 1541.9280663	total: 22.8s	remaining: 0us


1617.7176050557227

In [32]:
# Definir el modelo XGBoost
xgboost_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42, learning_rate=0.1)

# Entrenar el modelo de XGBoost
xgboost_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_xgb = xgboost_model.predict(X_test)

# Calcular el RMSE para el modelo XGBoost
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
rmse_xgb

1709.2603888157466

El modelo de regresión lineal es extremadamente rápido tanto para entrenar como para realizar predicciones. Sin embargo, su simplicidad también limita la calidad de las predicciones.
Los modelos avanzados de boosting, tienden a ser más lentos tanto en entrenamiento como en predicciones, pero generalmente ofrecen una mayor precisión.

Los modelos de boosting como LightGBM, CatBoost, y XGBoost requieren más tiempo para entrenar, especialmente si estamos ajustando los hiperparámetros. Es importante encontrar un equilibrio entre la precisión y el tiempo de entrenamiento.

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Excelente iniciativa al probar diferentes modelos, incluyendo regresión lineal y algoritmos de boosting como LightGBM, CatBoost y XGBoost. Esto te permite comparar su rendimiento y elegir el más adecuado para el problema.
</div>


<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
Aunque calculaste el RMSE para cada modelo, sería beneficioso realizar una validación cruzada para obtener una estimación más robusta del rendimiento del modelo y evitar posibles sesgos debidos a la división de datos.
</div>


<div class="alert alert-block alert-warning"> 
<b>Comentario del revisor</b> <a class="tocSkip"></a> 

Para mejorar tu proyecto y prepararlo para incluirlo en tu portafolio, te recomiendo que agregues una sección donde analices la **velocidad y la calidad de los modelos**. Esto es especialmente importante, ya que en aplicaciones del mundo real, no solo importa la precisión de las predicciones sino también la eficiencia en términos de tiempo de entrenamiento y predicción.

Puedes utilizar las **"magic cells"** de Jupyter Notebook para medir el tiempo de ejecución de tu código. Estas son herramientas muy útiles para obtener información sobre el rendimiento de tus celdas. Aquí te explico cómo puedes utilizarlas:

- **`%time`:** Esta magic line te permite medir el tiempo de ejecución de una sola línea de código.
  
  ```python
  %time model.fit(X_train, y_train)
    
%%time: Esta magic cell se coloca al inicio de una celda y mide el tiempo total de ejecución de toda la celda.

    %%time

    # Código para entrenar el modelo
    model.fit(X_train, y_train)
    
    
%timeit y %%timeit: Estas magic functions ejecutan el código varias veces para obtener un tiempo promedio y más preciso.


    %timeit model.predict(X_test)
    
Al aplicar estas herramientas, podrías:

Medir el tiempo de entrenamiento: Cuánto tiempo tarda cada modelo en ajustarse a los datos de entrenamiento.

Medir el tiempo de predicción: Cuánto tiempo toma cada modelo en realizar predicciones sobre el conjunto de prueba.

Comparar los resultados: Crear una tabla o resumen donde compares la calidad (por ejemplo, RMSE) y los tiempos de entrenamiento y predicción de cada modelo. Esto te permitirá evaluar cuál modelo es el más adecuado considerando tanto la precisión como la eficiencia.



</div> 

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

¡Felicidades! Tu proyecto está **aprobado**. Has demostrado una sólida comprensión del proceso de modelado y análisis de datos. A continuación, destaco los puntos positivos y algunas sugerencias para seguir mejorando:

#### Puntos Positivos:

- **Preparación de datos exhaustiva:** Has realizado un buen trabajo en la limpieza y preparación de los datos, identificando columnas irrelevantes y manejando valores faltantes y atípicos.

- **Exploración de múltiples modelos:** La implementación y comparación de diferentes algoritmos de regresión demuestra tu interés en encontrar la mejor solución para el problema.

- **Análisis crítico de resultados:** Has evaluado adecuadamente el equilibrio entre la precisión del modelo y el tiempo de entrenamiento, lo cual es esencial en proyectos del mundo real.

#### Sugerencias para Mejorar:

- **Codificación de variables categóricas:** Considera utilizar técnicas más adecuadas para variables categóricas, como `OneHotEncoder` o el manejo nativo de categorías en CatBoost y LightGBM, para evitar introducir relaciones ordinales donde no las hay.

- **Optimización de hiperparámetros:** Implementa técnicas de optimización como `GridSearchCV` o `RandomizedSearchCV` para ajustar los hiperparámetros de tus modelos y potencialmente mejorar su rendimiento.

- **Validación cruzada:** Utiliza validación cruzada para obtener estimaciones más fiables del rendimiento de tus modelos y asegurarte de que generalizan bien a datos nuevos.

- **Análisis de outliers adicionales:** Realiza un análisis más profundo de los valores atípicos en variables como `RegistrationYear` y `Power` para limpiar aún más los datos y mejorar la calidad del modelo.

¡Continúa con el excelente trabajo! Tu dedicación y esfuerzo son evidentes, y estás en el camino correcto para convertirte en un gran científico de datos. 🚀

</div>
